In [ ]:
#| default_exp core

In [ ]:
import gym
import d3rlpy
from d3rlpy.datasets import get_cartpole
from d3rlpy.algos import DiscreteCQL
from d3rlpy.metrics.scorer import discounted_sum_of_advantage_scorer
from d3rlpy.metrics.scorer import evaluate_on_environment
from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from sklearn.model_selection import train_test_split
import glob
import io
import base64

from gym.wrappers import RecordVideo
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display



/home/meherkh/anaconda3/envs/r_python_prod3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load cartpole dataset

In [ ]:
dataset, env = get_cartpole()



Setup data-driven deep reinforcement learning algorithm.


In [ ]:
# setup CQL algorithm
cql = DiscreteCQL(use_gpu=False)

# split train and test episodes
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)

# start training
cql.fit(
    train_episodes,
    eval_episodes=test_episodes,
    n_epochs=1,
    scorers={
            'environment': evaluate_on_environment(env), # evaluate with CartPol-v0 environment
            'advantage': discounted_sum_of_advantage_scorer, # smaller is better
            'td_error': td_error_scorer, # smaller is better
            'value_scale': average_value_estimation_scorer # smaller is better
        })

2022-12-21 21:51.20 [debug    ] RoundIterator is selected.
2022-12-21 21:51.20 [info     ] Directory is created at d3rlpy_logs/DiscreteCQL_20221221215120
2022-12-21 21:51.20 [debug    ] Building models...
2022-12-21 21:51.20 [debug    ] Models have been built.
2022-12-21 21:51.20 [info     ] Parameters are saved to d3rlpy_logs/DiscreteCQL_20221221215120/params.json params={'action_scaler': None, 'alpha': 1.0, 'batch_size': 32, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 0.99, 'generated_maxlen': 100000, 'learning_rate': 6.25e-05, 'n_critics': 1, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'q_func_factory': {'type': 'mean', 'params': {'share_encoder': False}}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': None, 'target_update_interval': 8000, 'use_gpu': None, 'algorithm': 'DiscreteCQL', 'observation

Epoch 1/1:   0%|               | 0/2488 [00:00<?, ?it/s]/home/meherkh/anaconda3/envs/r_python_prod3/lib/python3.9/site-packages/torch/autograd/__init__.py:154: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180549130/work/c10/cuda/CUDAFunctions.cpp:112.)
  Variable._execution_engine.run_backward(
Epoch 1/1: 100%|█| 2488/2488 [00:05<00:00, 434.93it/s, l


2022-12-21 21:51.27 [info     ] DiscreteCQL_20221221215120: epoch=1 step=2488 epoch=1 metrics={'time_sample_batch': 7.502825697135312e-05, 'time_algorithm_update': 0.0020884720075552104, 'loss': 0.6844233163060482, 'time_step': 0.002275211251433639, 'environment': 200.0, 'advantage': -3.0701209005002412, 'td_error': 1.1680058260126593, 'value_scale': 1.0348603269587753} step=2488
2022-12-21 21:51.27 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20221221215120/model_2488.pt


[(1,
  {'time_sample_batch': 7.502825697135312e-05,
   'time_algorithm_update': 0.0020884720075552104,
   'loss': 0.6844233163060482,
   'time_step': 0.002275211251433639,
   'environment': 200.0,
   'advantage': -3.0701209005002412,
   'td_error': 1.1680058260126593,
   'value_scale': 1.0348603269587753})]

Visualize episode training

In [ ]:
# start virtual display
display = Display(visible=0, size=(1400, 900))
display.start()

<pyvirtualdisplay.display.Display>

In [ ]:
# play recorded video
def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''
            
                {0}" type="video/mp4" />
            '''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")

In [ ]:
# wrap RecordVideo wrapper
env = RecordVideo(env, './video')

# evaluate
evaluate_on_environment(env)(cql)

/home/meherkh/anaconda3/envs/r_python_prod3/lib/python3.9/site-packages/gym/wrappers/record_video.py:41: UserWarning: WARN: Overwriting existing videos at /home/meherkh/Meher/deep_reinforcement_learning_hands_on/nbs/video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


200.0

In [ ]:
show_video()